In [644]:
import calendar
import datetime
from dateutil.relativedelta import relativedelta
from datetime import timedelta
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs

def getFirstAndLastDay(year,month):
    weekDay,monthCountDay = calendar.monthrange(int(year),int(month))
    # 获取当前月份最后一天
    lastDay = datetime.date(year,month,day=monthCountDay)
    # 返回第一天和最后一天
    if lastDay.isoweekday()==6:
        lastDay=lastDay- timedelta(days=1) 
    if lastDay.isoweekday()==7:
        lastDay=lastDay- timedelta(days=2)
    
        
    return lastDay.strftime("%Y%m%d")

In [645]:
def get_etf():
    etf_csv=pd.read_csv('Developed Asia Pacific ETF List (108).csv')
    symbol=etf_csv['Symbol']
    inception=etf_csv['Inception']
    count=0
    real_symbol=[]
    in_s=[]
    for candiate in inception:
        s=datetime.datetime.strptime(candiate, "%d/%m/%Y")
        if s<=datetime.datetime.strptime("31/12/2015", "%d/%m/%Y"):
           # print(s.strftime("%d/%m/%Y"))
            in_s.append(s.strftime("%d/%m/%Y"))
            real_symbol.append(symbol[count])
        count=count+1
    return real_symbol

In [646]:
def get_url(etf):
    url=[]  
    url.append("https://www.moneydj.com/ETF/X/xdjbcd/Basic0003BCD.xdjbcd?etfid="+etf+"&b=2015-12-31&c=2018-12-31")
    return url

In [648]:
# urls=get_etf()
# for url in urls:
#     print(get_url(url))

In [551]:
start_date="2015-12-31"
end_date="2018-12-31"
start = datetime.datetime.strptime(start_date, "%Y-%m-%d") # datetime.strptime(date_string, format) return datetime
stop = datetime.datetime.strptime(end_date, "%Y-%m-%d")

In [552]:
# end_date=[]
# while start<=stop:
#     year=start.strftime('%Y')
#     month=start.strftime('%m')
#     year=int(year)
#     month=int(month)
#     print(getFirstAndLastDay(year,month))
#     end_date.append(getFirstAndLastDay(year,month))
#     start+=relativedelta(months=1)

In [553]:
# end_date=[]

# while start<=stop:
#     beforeday=start
#     start+=timedelta(days=1)
#     nextday=start
#     if (beforeday.year and beforeday.month)!=(nextday.year and nextday.month):
#         print(beforeday)


In [658]:
def get_nav(url):
    res = requests.get(url) 
    doc = bs(res.text, 'lxml')
    data = doc.select_one('p').get_text()
    datas=data.split(' ',1)
    date=datas[0]
    date=date.split(',')
    days=[] #csv裡面的date
    for day in date:
        day=datetime.datetime.strptime(day, "%Y%m%d")
        days.append(day)
    end_day=[] #月底
    for i in range(len(days)-1):
        beforeday=days[i]
        nextday=days[i+1]
        if (beforeday.year and beforeday.month)!=(nextday.year and nextday.month):
            end_day.append(beforeday.strftime("%Y%m%d"))
    end_day.append(days[len(days)-1].strftime("%Y%m%d"))# last day
#     print(end_day[36])
#     print(len(end_day))
    value=datas[1].replace(' ',',')
    value=value.split(',')
    v1=value[0:755]
    index=[]
    nav=[]
    for i in range(0,37):
#         print(end_day[i])
        count=0
        for   cday in date:
            if end_day[i]==cday:
                index.append(count)
                nav.append(v1[count])
            count=count+1
#     print(index)
#     print(nav)
    return end_day,nav
    


In [662]:
end_day,nav=get_nav("https://www.moneydj.com/ETF/X/xdjbcd/Basic0003BCD.xdjbcd?etfid=EWY&b=2015-12-31&c=2018-12-31")

In [664]:
print(len(end_day),len(nav))

37 37


In [668]:

dict={
        "date":end_day,
        "index":index,
        "nav":nav
}
data=pd.DataFrame(dict)
data

,date,index,nav
0,20151231,0,50.4300
1,20160129,19,47.6800
2,20160229,39,46.5000
3,20160331,61,52.9000
4,20160429,82,52.7600
5,20160531,103,50.1900
6,20160630,125,52.0800
7,20160729,145,55.2100
8,20160831,168,56.9000
9,20160930,189,57.7300


In [652]:
print(v2)

['48.4800', '47.7200', '48.3200', '47.4800', '46.7600', '45.7600', '46.0800', '45.8400', '45.4400', '46.0000', '44.7000', '45.0800', '43.3600', '43.2000', '45.1600', '44.2000', '44.8000', '44.8400', '44.9200', '46.0000', '45.7200', '45.2400', '44.6800', '44.5600', '43.8000', '43.7200', '42.8800', '42.1200', '41.2800', '41.6400', '43.1600', '43.5200', '43.5200', '43.3200', '44.0400', '43.3200', '43.7600', '44.2800', '44.2400', '43.5600', '44.6400', '45.2800', '45.6000', '46.0800', '45.6000', '45.1600', '45.1200', '44.9200', '46.1200', '46.3200', '45.7200', '45.8000', '45.9600', '45.8000', '46.0000', '46.2800', '45.6800', '45.4000', '46.0800', '46.5600', '46.4000', '45.6400', '44.5200', '44.5200', '43.5600', '44.2800', '44.0400', '45.3200', '45.0400', '46.0800', '47.3600', '47.5600', '47.1200', '47.4800', '48.2000', '48.4800', '48.4800', '48.8400', '48.5200', '48.3200', '48.2800', '46.0800', '45.6800', '46.5200', '46.0800', '45.6800', '45.9200', '46.1600', '46.1200', '47.2800', '46.4000'

In [559]:
datas=data.split(' ',1)
date=datas[0]
date=date.split(',')
print(date[len(date)-1])
print(len(date))
# for i in date:
#     print(i)

20181231
755


In [579]:
days=[]
for day in date:
    day=datetime.datetime.strptime(day, "%Y%m%d")
    days.append(day)

In [577]:
end_day=[]
for i in range(len(days)-1):
    beforeday=days[i]
    nextday=days[i+1]
    if (beforeday.year and beforeday.month)!=(nextday.year and nextday.month):
        end_day.append(beforeday.strftime("%Y%m%d"))
end_day.append(days[len(days)-1].strftime("%Y%m%d"))
print(end_day[36])
print(len(end_day))

20181231
37


In [597]:
value=datas[1].replace(' ',',')
value=value.split(',')
print(len(value))
v2=value[755:]
v1=value[0:755]
print(v1)
# print(v2)


1510
['49.1200', '48.3200', '48.1900', '47.8500', '47.0200', '46.8700', '47.0100', '45.3400', '46.5000', '45.4200', '45.7400', '45.0600', '43.9500', '42.3600', '44.2800', '44.7900', '43.7900', '44.9800', '44.6500', '45.0700', '46.0600', '46.0100', '45.3400', '45.1700', '44.5600', '45.4200', '43.2000', '42.0200', '43.0100', '40.3700', '43.5200', '42.7800', '44.0000', '43.6800', '43.8400', '43.9300', '44.0500', '44.2100', '43.8600', '43.7800', '43.6000', '45.1800', '46.0100', '46.1000', '45.7200', '45.6700', '44.9900', '45.4200', '45.6100', '46.2400', '46.2200', '45.4900', '46.3900', '45.8300', '45.7400', '46.5300', '45.9500', '45.5900', '46.3300', '46.4900', '46.0900', '45.8600', '44.2800', '44.7300', '43.9100', '44.0700', '45.0900', '45.4000', '45.3200', '45.7600', '46.6600', '48.1000', '47.9300', '46.4000', '47.7100', '47.8700', '48.8100', '48.4600', '48.5500', '48.1000', '47.7200', '47.2800', '47.9700', '46.6700', '46.8100', '46.5100', '46.3000', '46.5300', '46.0800', '46.7600', '46.

In [599]:
index=[]
nav=[]
for i in range(0,37):
    print(end_day[i])
    count=0
    for   cday in date:
        if end_day[i]==cday:
            index.append(count)
            nav.append(v1[count])
        count=count+1
print(index)
print(nav)

20151231
20160129
20160229
20160331
20160429
20160531
20160630
20160729
20160831
20160930
20161031
20161130
20161230
20170131
20170228
20170331
20170428
20170531
20170630
20170731
20170831
20170929
20171031
20171130
20171229
20180131
20180228
20180329
20180430
20180531
20180629
20180731
20180831
20180928
20181031
20181130
20181231
[0, 19, 39, 61, 82, 103, 125, 145, 168, 189, 210, 231, 252, 272, 291, 314, 333, 355, 377, 397, 420, 440, 462, 483, 503, 524, 543, 564, 585, 607, 628, 649, 672, 691, 714, 735, 754]
['49.1200', '45.0700', '43.7800', '45.8600', '47.9700', '47.4700', '45.9200', '48.9000', '49.0500', '49.8200', '50.4700', '49.2300', '49.1100', '50.9200', '51.4600', '51.2500', '51.8100', '53.3400', '53.5500', '54.6100', '54.5700', '55.6400', '58.1800', '59.9000', '59.8900', '62.6100', '61.6400', '59.9100', '60.7500', '60.1200', '58.1300', '58.3400', '58.4500', '60.2300', '55.1200', '55.3300', '51.2000']


In [619]:

dict={
    "date":end_day,
    "index":index,
    "nav":nav
}
data=pd.DataFrame(dict)
data

,date,index,nav
0,20151231,0,49.1200
1,20160129,19,45.0700
2,20160229,39,43.7800
3,20160331,61,45.8600
4,20160429,82,47.9700
5,20160531,103,47.4700
6,20160630,125,45.9200
7,20160729,145,48.9000
8,20160831,168,49.0500
9,20160930,189,49.8200


In [621]:
data.to_csv(r'./nav.csv',index=False,sep=',')